#STAT 480 Fall 2019 Individual Project
### Name : Tianyi Li 
### NetID : tli76
### Project Title : Beer Recommendation

### Introduction
##### Many online businesses rely on customer reviews and ratings. It is especially important in the ecommerce industry where customer engagements are greatly impacted by ratings. Websites rely on rating data to power its recommendation engine to provide the best recommendations that are personalized and most relevant to the user and make profits. 
##### This project focuses on the beer industry/beer online shops. To recommend the users with their potential ideal product based on their past ratings or features of the beers will be the key motivation.

#### Import libraries

In [4]:
import pyspark.sql.functions as F
from pyspark.sql.types import *
from pyspark.sql.window import Window 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import math
import os
os.environ["PYSPARK_PYTHON"] = "python3"
import urllib
from pyspark.sql import SparkSession
## Recommendation Engine 
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS as ml_als
from pyspark.ml.tuning import ParamGridBuilder, TrainValidationSplit

#### Data ETL

In [6]:
## setup spark session
spark = SparkSession \
    .builder \
    .appName("beer review") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [7]:
## Create Dataframe and SQL Table
## load data into dataframe and create sql tables
beers = spark.read.load("/FileStore/tables/beer_reviews.csv", format='csv', header = True)
beers.createOrReplaceTempView("beer_reviews")

In [8]:
## Display raw data
display(beers.take(5))

brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
10325,Vecchio Birraio,1234817823,1.5,2,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5,47986
10325,Vecchio Birraio,1235915097,3,2.5,3,stcules,English Strong Ale,3,3,Red Moon,6.2,48213
10325,Vecchio Birraio,1235916604,3,2.5,3,stcules,Foreign / Export Stout,3,3,Black Horse Black Beer,6.5,48215
10325,Vecchio Birraio,1234725145,3,3,3.5,stcules,German Pilsener,2.5,3,Sausa Pils,5,47969
1075,Caldera Brewing Company,1293735206,4,4.5,4,johnmichaelsen,American Double / Imperial IPA,4,4.5,Cauldron DIPA,7.7,64883


#### In this project, 1.5 millions of beer review data accessed through Kaggle will be analyzed.
#### Reviewer information, beer Information and rating information are provided. 
##### Specifically, 
##### Beer information includes : Beer Name, Beer ABV, Beer ID, Beer Style, Brewery Name, Brewery ID.
##### Rating infomration includes : Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings.
##### Reviewer information includes : Reviewer profilename, Review time.

In [10]:
## Show data types
beers.printSchema()

root
-- brewery_id: string (nullable = true)
-- brewery_name: string (nullable = true)
-- review_time: string (nullable = true)
-- review_overall: string (nullable = true)
-- review_aroma: string (nullable = true)
-- review_appearance: string (nullable = true)
-- review_profilename: string (nullable = true)
-- beer_style: string (nullable = true)
-- review_palate: string (nullable = true)
-- review_taste: string (nullable = true)
-- beer_name: string (nullable = true)
-- beer_abv: string (nullable = true)
-- beer_beerid: string (nullable = true)

In [11]:
## Convert from spark dataframe to pandas dataframe
pandasbeer = beers.toPandas()

/databricks/spark/python/pyspark/sql/types.py:1636: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
 return pa.schema(fields)
/databricks/python/lib/python3.7/site-packages/pyarrow/__init__.py:152: UserWarning: pyarrow.open_stream is deprecated, please use pyarrow.ipc.open_stream
 warnings.warn("pyarrow.open_stream is deprecated, please use "

In [12]:
## Change reviewer profilename to userid
listOfStr = pandasbeer['review_profilename'].tolist()
my_dict = { i: listOfStr[i] for i in range(0, len(listOfStr) )}
flipped_dict = dict(zip(my_dict.values(), my_dict.keys()))
pandasbeer['review_profilename'] = pandasbeer.review_profilename.map(flipped_dict)

In [13]:
## Convert back to spark dataframe
mybeer = spark.createDataFrame(pandasbeer)

### Exploratory data analysis
#### In the first part of this project, I am going to explore what kind of data we get. 
##### I also focused on beer style, since it might be an interesting characteristic that affects people's choice of beer, thus the ratings of beer.

In [15]:
## Size of distinct items
print ("number of distinct users", mybeer.select('review_profilename').distinct().count())
print ("number of distinct beers", mybeer.select('beer_name').distinct().count())
print ("number of distinct beer styles", mybeer.select('beer_style').distinct().count())
print ("number of distinct breweries", mybeer.select('brewery_name').distinct().count())

number of distinct users 33388
number of distinct beers 56857
number of distinct beer styles 104
number of distinct breweries 5743

#### Explore beer styles

In [17]:
## Unique Beer Styles
unique_beer_style = mybeer.select('beer_style').distinct()
display(unique_beer_style)

beer_style
Baltic Porter
Milk / Sweet Stout
American Dark Wheat Ale
American Amber / Red Lager
Dubbel
Dortmunder / Export Lager
Tripel
Belgian Strong Dark Ale
Belgian Strong Pale Ale
Wheatwine


In [18]:
## The number of type of beers in each beer style
df_style= mybeer \
                    .groupBy('beer_style').count() \
                    .orderBy('count', ascending = False)
display(df_style)

beer_style,count
American IPA,117586
American Double / Imperial IPA,85977
American Pale Ale (APA),63469
Russian Imperial Stout,54129
American Double / Imperial Stout,50705
American Porter,50477
American Amber / Red Ale,45751
Belgian Strong Dark Ale,37743
Fruit / Vegetable Beer,33861
American Strong Ale,31945


In [19]:
##Example of beers of belgian strong dark ale style
display(mybeer.where("beer_style like '%Belgian Strong Dark Ale%'"))

brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,beer_name,beer_abv,beer_beerid
1075,Caldera Brewing Company,1310518783,4,4,4.5,1585716,Belgian Strong Dark Ale,4,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1326071409,3.5,3.5,4,1581037,Belgian Strong Dark Ale,2,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1321407186,3,4,2,1584275,Belgian Strong Dark Ale,3,2.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1311736668,4,4,4,1584282,Belgian Strong Dark Ale,3.5,4,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1321070530,3.5,3.5,3.5,1584289,Belgian Strong Dark Ale,3,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1320555535,4,3.5,4,1574796,Belgian Strong Dark Ale,3.5,4,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1319921102,3.5,3.5,4,1582325,Belgian Strong Dark Ale,3.5,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1319858005,3.5,3.5,4,1577029,Belgian Strong Dark Ale,4,3.5,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1317515953,4,3,4,1585696,Belgian Strong Dark Ale,3.5,4,Vas Deferens Ale,8.1,58920
1075,Caldera Brewing Company,1317093500,3,3.5,4,1579574,Belgian Strong Dark Ale,3.5,3.5,Vas Deferens Ale,8.1,58920


#### Explore the overall ratings of beers

In [21]:
## The number of ratings of each scores
df_rate= mybeer \
                    .groupBy('review_overall').count() \
                    .orderBy('review_overall', ascending = False)
display(df_rate)

review_overall,count
5,91320
4.5,324385
4,582764
3.5,301817
3,165644
2.5,58523
2,38225
1.5,12975
1,10954
0,7


#### The average rating of beers from each beer style

In [23]:
pandasbeer['review_overall'] = pandasbeer['review_overall'].apply(pd.to_numeric)
pandasbeer.groupby("beer_style")["review_overall"].mean().sort_values(ascending = False).head(10)

Out[15]: beer_style
American Wild Ale 4.093262
Gueuze 4.086287
Quadrupel (Quad) 4.071630
Lambic - Unblended 4.048923
American Double / Imperial Stout 4.029820
Russian Imperial Stout 4.023084
Weizenbock 4.007969
American Double / Imperial IPA 3.998017
Flanders Red Ale 3.992722
Rye Beer 3.981737
Name: review_overall, dtype: float64

#### The beers that have the most ratings

In [25]:
beerrated = pandasbeer.groupby('beer_name')['review_overall'].count().sort_values(ascending=False).head(5)

#### The number of ratings that each reviewer gave

In [27]:
pandasbeer.groupby('review_profilename')['review_overall'].count().sort_values(ascending=False).head(5) 

Out[18]: review_profilename
1586535 5817
1586244 4661
1586246 4617
1584562 3518
1586447 3497
Name: review_overall, dtype: int64

##### Although using this interesting dataset from BeerAdvocate, the project can go into many directions.
##### In the rest part of this project, I am going to dive directly into predicting beer ratings only from review ID, and ratings that they gave to other beers.

### Start to predict beer ratings using recommendation algorithms with Spark MLlib APIs

In [30]:
## Create a new dataframe only focusing on the beer, reviewer and the ratings that give.
df_rating_data = mybeer.select("review_profilename","beer_beerid","review_overall")
df_rating_data = df_rating_data \
            .withColumn("review_profilename", df_rating_data.review_profilename.cast(IntegerType())) \
            .withColumn("beer_beerid", df_rating_data.beer_beerid.cast(IntegerType())) \
            .withColumn("review_overall", df_rating_data.review_overall.cast(DoubleType())) 

In [31]:
## Train and test split
data, hold_out = df_rating_data.randomSplit([0.8, 0.2], seed = 7856)
data.cache()
hold_out.cache()

Out[31]: DataFrame[review_profilename: int, beer_beerid: int, review_overall: double]

#### Use ALS (Alternating Least Square) and collaborative filtering to predict the ratings for the movies
#### ALS machine learning model referred from 
##### https://spark.apache.org/docs/2.2.0/ml-collaborative-filtering.html

In [33]:
# Build the recommendation model using ALS on the training data

# Specify model with parameters
als = ml_als(userCol="review_profilename", itemCol="beer_beerid", ratingCol="review_overall", coldStartStrategy="drop")

# Use a ParamGridBuilder to construct a grid of parameters to search over.
paramGrid = ParamGridBuilder()\
    .addGrid(als.maxIter, [10]) \
    .addGrid(als.rank, [6, 8, 10, 12, 14])\
    .addGrid(als.regParam, [0.05, 0.1, 0.2, 0.4, 0.8])\
    .build()

# Evaluate the model by computing the RMSE on the test data
evaluator = RegressionEvaluator(metricName="rmse", labelCol="review_overall",
                                predictionCol="prediction")

# Then we do a parameter grid search (regularization parameter and rank), and use validation set performance to find the optimal set of hyperparameter. Do cross validation for using all parameters, but only do train-test split once for data.
tvs = TrainValidationSplit(estimator=als,
                           estimatorParamMaps=paramGrid,
                           evaluator=evaluator,
                           trainRatio=0.8)

# fit data
myalsmodel = tvs.fit(data)

MLlib will automatically track trials in MLflow. After your tuning fit() call has completed, view the MLflow UI to see logged runs.

#### Evaluate the model by computing the RMSE on the test data

In [35]:
predictions = myalsmodel.transform(hold_out)
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 0.6194330698473862

#### Show predicted ratings

In [37]:
## Show predictions
display(predictions)

review_profilename,beer_beerid,review_overall,prediction
1577591,148,4.0,3.8799534
1551381,148,3.5,3.8141747
1527349,148,4.5,3.7799087
1571478,148,3.5,3.7675872
1582002,148,4.5,3.9979942
1586244,148,4.5,3.9629683
1527086,148,4.0,4.04106
1582200,148,4.0,3.9003925
1570109,148,4.0,3.9889219
1585322,148,3.0,3.3367622


#### Generate the top 5 beer recommendations for each user
#### Generate the top 5 user recommendations for each beer

In [39]:
# Generate top 5 beer recommendations for each user
userRecs = myalsmodel.bestModel.recommendForAllUsers(5).cache()
display(userRecs)

review_profilename,recommendations
6466,"List(List(32744, 5.073592), List(42994, 5.0728498), List(49042, 5.0502815), List(19073, 5.042628), List(67231, 4.9889407))"
16503,"List(List(32744, 5.4413967), List(49042, 5.4118657), List(42994, 5.38119), List(19073, 5.3472524), List(52165, 5.331808))"
22373,"List(List(32744, 5.854742), List(76354, 5.8357277), List(51494, 5.8105693), List(36423, 5.780042), List(35361, 5.773715))"
29814,"List(List(32744, 4.497129), List(49042, 4.457826), List(42994, 4.448377), List(51494, 4.4227533), List(66841, 4.419847))"
83242,"List(List(12875, 6.6870155), List(35361, 6.432667), List(36423, 6.410732), List(76354, 6.3860884), List(32744, 6.381494))"
83250,"List(List(12875, 5.349613), List(35361, 5.146133), List(36423, 5.1285853), List(76354, 5.10887), List(32744, 5.105195))"
111515,"List(List(19073, 7.209939), List(52165, 7.1285963), List(56839, 7.1146684), List(49042, 7.108617), List(32744, 7.0716853))"
227038,"List(List(32744, 6.2366395), List(76354, 6.234305), List(35361, 6.2007565), List(36423, 6.1998997), List(51494, 6.18927))"
234425,"List(List(32744, 6.7871356), List(49042, 6.7788973), List(52165, 6.7439585), List(19073, 6.738013), List(35361, 6.7369223))"
244629,"List(List(35361, 2.6083317), List(66599, 2.5542696), List(20326, 2.551909), List(73376, 2.5372114), List(34345, 2.511363))"


In [40]:
# Generate top 5 user recommendations for each beer
movieRecs = myalsmodel.bestModel.recommendForAllItems(5).cache()
display(movieRecs)

beer_beerid,recommendations
148,"List(List(1312486, 7.5216894), List(1427753, 6.6929045), List(1024174, 6.552963), List(363324, 6.0417175), List(1433900, 6.01709))"
463,"List(List(1024174, 5.6211762), List(1312486, 5.573538), List(1427753, 5.387783), List(363324, 4.944876), List(1299268, 4.7739987))"
496,"List(List(1312486, 8.204102), List(1427753, 7.4913464), List(1024174, 7.1929536), List(363324, 6.8434496), List(1433900, 6.7891383))"
833,"List(List(1312486, 6.677062), List(1024174, 6.157333), List(1427753, 6.1311717), List(363324, 5.5679755), List(847194, 5.555277))"
1088,"List(List(1312486, 7.675696), List(1427753, 6.6040287), List(1024174, 6.2693977), List(1433900, 6.1197863), List(363324, 6.0172606))"
1238,"List(List(1427753, 6.7765694), List(1024174, 6.773761), List(1312486, 6.7403274), List(363324, 6.548501), List(316507, 6.141679))"
1580,"List(List(1427753, 6.277424), List(1312486, 6.2224917), List(1024174, 5.8667927), List(363324, 5.7274847), List(1304602, 5.6583815))"
1591,"List(List(1312486, 6.5206647), List(1427753, 5.794325), List(1024174, 5.6321497), List(1433900, 5.3694997), List(363324, 5.3452187))"
1645,"List(List(1312486, 6.666706), List(1427753, 5.873096), List(1024174, 5.8699174), List(847194, 5.4195747), List(1433900, 5.335081))"
1959,"List(List(1312486, 6.6604233), List(1427753, 6.419442), List(1024174, 6.1746483), List(363324, 5.979307), List(1433900, 5.8491807))"


### Discussion
#### After running our fitted model on test data, the RMSE is 0.6. The result is generally satisfying.
#### The predicted beer ratings and recommendations for customers are also shown above, providing recommended beers to the specific customers.
#### More explorations could be done using this dataset in possible future projects.
##### For example, to figure out how we could use peoples' ratings for aroma, palate, taste and appearence to predict thier overall ratings for the beer.
##### Or to predict customers' preferences of breweries and so on.

### Appendix for all the columns in the dataset 
##### Reviewer Information : Reviewer profilename, Review time
###### review_profilename 
###### review_time
##### Rating Information : Overall Ratings, Aroma Ratings, Apprearence Ratings, Palate Ratings, Taste Ratings
###### review_overall
###### review_aroma
###### review_appearance
###### review_palate
###### review_taste
##### Beer Information: Beer Name, Beer ABV, Beer ID, Beer Style, Brewery Name, Brewery ID
###### beer_name
###### beer_abv
###### beer_beerid
###### beer_style
###### brewery_name
###### brewery_id

### References 
##### https://www.kaggle.com/rdoume/beerreviews
##### https://hub.packtpub.com/building-recommendation-engine-spark/ 
##### https://www.analyticsvidhya.com/blog/2016/06/quick-guide-build-recommendation-engine-python/ 
##### https://blog.statsbot.co/recommendation-system-algorithms-ba67f39ac9a3